In [117]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import json
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L-12-v2')

loading configuration file https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-12-v2/resolve/main/config.json from cache at /Users/joshuamin/.cache/huggingface/transformers/e7a41a6174913880df8e829b641066e966206fab8954ee632010d4f61628ae02.f0aa8e636bcccae4be6e2271678827e07b888454c0bc3f0a300929ab2bc621b3
Model config BertConfig {
  "_name_or_path": "cross-encoder/ms-marco-MiniLM-L-12-v2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "sbert_ce_default_activat

In [10]:
s = open("../prompt_engineering/gpt-3_semantic_search/1_top_quality.json")
first = json.load(s)
s = open("../prompt_engineering/gpt-3_semantic_search/2_decent_enough_to_keep.json")
second = json.load(s)
s = open("../prompt_engineering/gpt-3_semantic_search/3_to_delete.json")
third = json.load(s)
s = open("../prompt_engineering/gpt-3_semantic_search/4_invalid_items.json")
fourth = json.load(s)

In [11]:
bad_data = []
for dataset in [third, fourth]:
    for row in dataset:
        bad_data.append(row['GPT-3-Semantic-Search-Generations']['answer'])

In [165]:
anchors = []
positives = []
negatives = []
all_data = []
all_dict = []
for dataset in [first, second]:
    for row in dataset:
        anchors.append(row['GPT-3-Semantic-Search-Generations']['question'])
        positives.append(row['GPT-3-Semantic-Search-Generations']['answer'])
        negatives.append(random.choice(bad_data))
        all_data.append([row['GPT-3-Semantic-Search-Generations']['question'], row['GPT-3-Semantic-Search-Generations']['answer'],random.choice(bad_data)])
        all_dict.append({'query': row['GPT-3-Semantic-Search-Generations']['question'], 'pos':row['GPT-3-Semantic-Search-Generations']['answer'], 'neg':random.choice(bad_data)})

In [166]:
train_dict, test_dict = train_test_split(all_dict, test_size=.2)
train_list, test_list = train_test_split(all_data, test_size=.2)

In [152]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [173]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [182]:
import torch

class filtered(torch.utils.data.Dataset):
    def __init__(self, encoding):
        self.encoding = encoding

    # def __getitem__(self, idx):
    #     item = {key: torch.tensor(val[idx]) for key, val in self.encoding.items()}
    #     item['labels'] = torch.tensor(self.labels[idx])
    #     return item

    def __len__(self):
        return len(self.encoding)

In [177]:
training_encoded = []
for row in train_list:
    tokenized = tokenizer(row, padding=True, truncation=True)
    training_encoded.append(tokenized)
testing_encoded = []
for row in test_list:
    tokenized = tokenizer(row, padding=True, truncation=True)
    testing_encoded.append(tokenized)

In [183]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=filtered(training_encoded),
    eval_dataset=filtered(testing_encoded),
    tokenizer=tokenizer,
)

In [184]:
trainer.train()

/Users/joshuamin/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 96
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 36


  0%|          | 0/36 [00:00<?, ?it/s]

NotImplementedError: 